<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DSE-203:-Final-Project" data-toc-modified-id="DSE-203:-Final-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DSE 203: Final Project</a></span><ul class="toc-item"><li><span><a href="#Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag" data-toc-modified-id="Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Author: Adelle Driker, Bo Yan, Paul Cabasag</a></span></li></ul></li><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Create-Functions" data-toc-modified-id="Create-Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create Functions</a></span></li><li><span><a href="#Declarations" data-toc-modified-id="Declarations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Declarations</a></span></li><li><span><a href="#Recipe-Data" data-toc-modified-id="Recipe-Data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Recipe Data</a></span><ul class="toc-item"><li><span><a href="#Get-recipe-file-from-Spoonacular-API" data-toc-modified-id="Get-recipe-file-from-Spoonacular-API-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Get recipe file from Spoonacular API</a></span></li><li><span><a href="#Convert-json-file-to-csv-file" data-toc-modified-id="Convert-json-file-to-csv-file-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Convert json file to csv file</a></span></li><li><span><a href="#Clean-up-batch-csv-files" data-toc-modified-id="Clean-up-batch-csv-files-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Clean up batch csv files</a></span></li><li><span><a href="#Combine-all-csv-files" data-toc-modified-id="Combine-all-csv-files-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Combine all csv files</a></span></li><li><span><a href="#Preprocess-Recipe-Data" data-toc-modified-id="Preprocess-Recipe-Data-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Preprocess Recipe Data</a></span></li><li><span><a href="#Output-Recipe-Data" data-toc-modified-id="Output-Recipe-Data-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Output Recipe Data</a></span></li></ul></li></ul></div>

# DSE 203: Final Project

## Author: Adelle Driker, Bo Yan, Paul Cabasag

# Import Libraries

In [1]:
import pandas as pd
import requests
import json
import os
import glob
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re

# Create Functions

In [2]:
def get_recipe(url, headers, query_string, index, file_type):
    response = requests.request("GET", url, headers = headers, params = query_string)
    file_name = 'spoonacular_recipe_list_{}.{}'.format(index, file_type)
    with open(file_name, 'wb') as outf:
        outf.write(response.content)

In [3]:
def convert_json_to_csv(index, file_type_1 = 'json', file_type_2 = 'csv'):    
    file_name_1 = 'spoonacular_recipe_list_{}.{}'.format(index, file_type_1)
    file_name_2 = 'new_spoonacular_recipe_list_{}.{}'.format(index, file_type_2)
    
    with open(file_name_1, encoding = 'utf-8-sig') as f_input:
        df = pd.read_json(f_input)

    df.to_csv(file_name_2, encoding = 'utf-8', index = True)

In [4]:
def clean_up_batch_csv_files(index, file_type = 'csv'):
    file_name = 'spoonacular_recipe_list_{}.{}'.format(index, file_type)
    data = pd.read_csv(file_name)
    
    df = pd.DataFrame(data)
    df.columns = df.columns.str.replace('recipes/', '')
    df.columns = df.columns.str.replace('/', '_')
    recipe_list = []
    
    for columnName in df.columns:
        if df.dtypes[columnName] == 'bool':
            recipe_list.append("%s: toBoolean(csvLine.%s)" % (columnName, columnName))
        if df.dtypes[columnName] == 'object':
            recipe_list.append("%s: csvLine.%s" % (columnName, columnName))
        if df.dtypes[columnName] == 'int64':
            recipe_list.append("%s: toInteger(csvLine.%s)" % (columnName, columnName))
        if df.dtypes[columnName] == 'float64':
            recipe_list.append("%s: toFloat(csvLine.%s)" % (columnName, columnName))
    
    df.to_csv(file_name)
    
    with open("recipe_list.txt", 'w') as output:
        for row in recipe_list:
            output.write(str(row) + ',\n')

In [5]:
def combine_all_csv_files(input_file_path, output_file_name):
    os.chdir(input_file_path)
    all_filenames = [i for i in glob.glob('spoonacular_recipe_list_{}.csv'.format(i))]
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
    combined_csv.to_csv(output_file_name, index = False, encoding = 'utf-8-sig')

# Declarations

In [10]:
url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/random"

In [11]:
headers = {
    'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
    'x-rapidapi-key': "3909245fe8msh2162ec791f4d00ap1e234ejsnf21796e5d332"
    }

In [12]:
query_string = {"tags": "", "number":"100"}

In [13]:
file_number = 79

In [14]:
test_number = 10

# Recipe Data

## Get recipe file from Spoonacular API

In [15]:
for i in range(test_number):
    get_recipe(url, headers, query_string, index = i, file_type = 'json')

## Convert json file to csv file

In [16]:
for i in range(test_number):
    convert_json_to_csv(index = i, file_type_1 = 'json', file_type_2 = 'csv')

## Clean up batch csv files

In [17]:
for i in range(test_number):
    clean_up_batch_csv_files(index = i, file_type = 'csv')

## Combine all csv files

In [18]:
input_file_path = "/Users/boyan/Documents/DSE 203"
output_file_name = "RecipeList.csv"
combine_all_csv_files(input_file_path, output_file_name)

In [19]:
RecipeList = pd.read_csv("RecipeList.csv")
RecipeList.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,analyzedInstructions_0_steps_12_equipment_2_id,analyzedInstructions_0_steps_12_equipment_2_name,analyzedInstructions_0_steps_12_equipment_2_localizedName,analyzedInstructions_0_steps_12_equipment_2_image,analyzedInstructions_0_steps_13_number,analyzedInstructions_0_steps_13_step,analyzedInstructions_0_steps_13_equipment_0_id,analyzedInstructions_0_steps_13_equipment_0_name,analyzedInstructions_0_steps_13_equipment_0_localizedName,analyzedInstructions_0_steps_13_equipment_0_image
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1,1,1,1,1,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,2,2,2,2,2,2,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,3,3,3,3,3,3,3,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,4,4,4,4,4,4,4,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preprocess Recipe Data

In [20]:
new_RecipeList = pd.DataFrame()
new_RecipeList = RecipeList[['id', 'title', 'summary', 'readyInMinutes', 'pricePerServing', 'glutenFree', 'dairyFree', 'vegan']].copy()

In [21]:
recipe_list = new_RecipeList.dropna(axis = 0)
recipe_list = new_RecipeList.drop_duplicates()
recipe_list

,id,title,summary,readyInMinutes,pricePerServing,glutenFree,dairyFree,vegan
0,635856,Braised Pork: Filipino Humba,If you have approximately <b>about 45 minutes<...,45,147.72,False,True,False
1,715407,Easy Banana Pudding,This recipe serves 5 and costs $2.24 per servi...,45,224.38,False,False,False
2,658007,Raw Vegan Chocolate and Raspberry Cake,If you have around <b>about 45 minutes</b> to ...,45,141.76,False,True,False
3,638779,Chobani Harvest Fruit Salad,If you have roughly <b>roughly 45 minutes</b> ...,45,979.48,True,False,False
4,715497,Berry Banana Breakfast Smoothie,If you have around <b>5 minutes</b> to spend i...,5,204.29,False,False,False
5,638604,Chilled Swiss Oatmeal,Chilled Swiss Oatmeal might be a good recipe t...,10,188.99,True,False,False
6,664473,Vegan Peanut Butter Chocolate Fudge,Vegan Peanut Butter Chocolate Fudge is a <b>gl...,45,55.88,True,True,True
7,638832,Chocolate Banoffee Pie,Chocolate Banoffee Pie might be just the desse...,240,214.61,False,False,False
8,639749,Coconut Cream Pie Vegan Milkshake,Coconut Cream Pie Vegan Milkshake requires rou...,45,81.11,False,True,True
9,794350,Cherry Coconut Milk Smoothie,Cherry Coconut Milk Smoothie might be just the...,45,203.80,True,True,True


## Output Recipe Data

In [22]:
recipe_list.to_csv("Recipe.csv")